<h1>CNNs for EPRV on HARPS</h1>
The goal here is to training a CNN using HARPS images to the outputs of the HARPS EPRV extraction pipeline to see it a large of NN can replicated more explicit modeling.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

import random
import numpy as np
torch.manual_seed(101101)
random.seed(101101)
np.random.seed(101101)

/ext3/miniconda3/envs/cnnenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /ext3/miniconda3/envs/cnnenv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import os
# os.environ['CUDA_VISIBLE_DEVICES']

In [3]:
print(torch.__version__)

1.11.0


In [4]:
print(torch.cuda.is_available())

True


In [5]:
torch.cuda.device_count()

1

In [6]:
import subprocess
subprocess.run(["python", "--version"])

Python 3.10.9


CompletedProcess(args=['python', '--version'], returncode=0)

In [7]:
import torchvision

In [8]:
print(torch.backends.cudnn.enabled)

True


In [9]:
subprocess.run(['nvcc','--version'])

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0


CompletedProcess(args=['nvcc', '--version'], returncode=0)

In [10]:
subprocess.run(['conda','list'])

# packages in environment at /ext3/miniconda3/envs/cnnenv:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                  2_kmp_llvm    conda-forge
alsa-lib                  1.2.8                h166bdaf_0    conda-forge
anyio                     3.6.2              pyhd8ed1ab_0    conda-forge
aom                       3.5.0                h27087fc_0    conda-forge
argon2-cffi               21.3.0             pyhd8ed1ab_0    conda-forge
argon2-cffi-bindings      21.2.0          py310h5764c6d_3    conda-forge
astropy                   5.2.1           py310h0a54255_0    conda-forge
asttokens                 2.2.1              pyhd8ed1ab_0    conda-forge
attr                      2.5.1                h166bdaf_1    conda-forge
attrs                     22.2.0             pyh71513ae_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-fo

CompletedProcess(args=['conda', 'list'], returncode=0)

<h2>Model definition</h2>

Higher stride at higher layers,

In [11]:
class DownSizeNet(nn.Module):
    def __init__(self, in_size, out_size, normalize=True,stride=1, padding=1, leaky_slope=0.2):
        super(DownSizeNet, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 3, stride=stride, padding=padding, bias=False).double()]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size, 0.8).double())
        layers.append(nn.LeakyReLU(leaky_slope).double())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    
class RV_Model(nn.Module):
    def __init__(self):
        super(RV_Model, self).__init__()
        self.down1 = DownSizeNet(3,    64,normalize=False,stride=1)
        self.down2 = DownSizeNet(64,  128,stride=1,padding=0)
        self.down3 = DownSizeNet(128, 256,stride=2,padding=0)
        self.down4 = DownSizeNet(256, 256,stride=2,padding=0)
        self.down5 = DownSizeNet(256, 256,stride=2)
        self.down6 = DownSizeNet(256, 256,stride=2)
        
        self.dense1 = nn.Sequential(nn.Linear(65536,256), nn.ReLU()).double()
        self.dense2 = nn.Sequential(nn.Linear(256,256), nn.ReLU()).double()
        
        self.final = nn.Linear(256,1).double()
        
        
    def forward(self, x):
        # Propogate noise through fc layer and reshape to img shape
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
#         print(d6.shape)
#         print(d4.shape)
        d6 = torch.flatten(d6,1)
#         print(d6.shape)
#         print(d6.shape)
        d7 = self.dense1(d6)
        d8 = self.dense2(d7)
        
        return self.final(d8)

In [12]:
from torch.utils.data import Dataset
import glob

In [13]:
from astropy.io import fits

<h2>Dataset importing</h2>
The question here is how is the data organized in the directory and how can it be imported with the target RV. 

Not the OG data but after the data is saved from the pre processing step.

In [14]:
import pickle

In [15]:
import numpy as np
import h5py as h5

256 by 17 with 16 overlap should be the best size cropping. shit shit shit
0-256 240-496 480-736 720-976 
what is wrong with me im dumb
so dumb dumb dumb

In [16]:
def crop_again(img):
    size = 96
    n    = 6
    tot  = img.shape[-1]
    diff = int(((size*n) - tot)/(n-1))
#     print(diff)
    split_img = np.empty((0,3,size,size))
    for i in range(n):
        for j in range(n):
            temp = img[...,(j*size - j*diff):((j+1)*size - j*diff),(i*size - i*diff):((i+1)*size - i*diff)]
#         print(temp.shape)
            split_img = np.append(split_img,temp[None,...],axis=0)
    return split_img

In [17]:
test = np.ones((3,526,526))
out = crop_again(test)

In [18]:
out.shape

(36, 3, 96, 96)

different array for

In [19]:
def h5_to_array(ds):
    rvs_stack = []
    
    img_stack = np.empty((0,3,526,526))
    for visit_name,vist_info in ds['visits'].items():
        for hdu_num in ds['images'][visit_name].keys():
            img   = np.array(ds['images'][visit_name][hdu_num])
            if np.sum(img.shape) != 526*2:
                pass
#                 print(np.sum(img.shape))
            if np.sum(img.shape) == (526*2):
                rvs_stack += [np.double(vist_info.attrs['ESO DRS CCF RVC'])]
                all_flats = np.empty([0,526,526])
                for key in vist_info.attrs.keys():
                    if vist_info.attrs[key] == 'FLAT':
                        temp = np.array(ds['images'][key][hdu_num])[None,...]
                        all_flats = np.append(all_flats, temp,axis=0)
                    if vist_info.attrs[key] == 'THAR_THAR':
                        cali = np.array(ds['images'][key][hdu_num])
#                 temp = crop_again(np.stack((img,np.median(all_flats,axis=0),cali)))
                temp = np.stack((img,np.median(all_flats,axis=0),cali))[None,...]
#                 print(temp.shape)
                img_stack = np.append(img_stack,temp,axis=0)
    return img_stack, np.array(rvs_stack)

In [20]:
import pickle
def save(filename,model):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

def load(filename):
    with open(filename, 'rb') as input:  # Overwrites any existing file.
        model = pickle.load(input)
        return model

In [21]:
import os.path as path

In [22]:
files1 = glob.glob('/scratch/mdd423/CNN_EPRV/data/peg51_256/*.h5')

crop median and sift through h5s here. then pickle as nd arrays

In [23]:
filename             = files1[0]
# ds                   = h5.File(filename,'r')
# img_stack, rvs_stack = h5_to_array(ds)

In [24]:
dir_name, tailname = path.split(filename)
tailname = tailname[:-3]
imgname = path.join(dir_name, tailname + '_img.nda')
rvsname = path.join(dir_name, tailname + '_rvs.nda')
bcsname = path.join(dir_name, tailname + '_bcs.nda')
timname = path.join(dir_name, tailname + '_tim.nda')
# print(imgname,rvsname)
# save(imgname,img_stack)
# save(rvsname,rvs_stack)
# save(bcsname,bcs_stack)
# save(timname,tim_stack)
img_stack = load(imgname)
rvs_stack = load(rvsname)
bcs_stack = load(bcsname)
tim_stack = load(timname)

In [25]:
class ND_Dataset(Dataset):
    def __init__(self, imgs,rvs):
        self.img_stack = imgs
        self.rvs_stack = rvs
        self.type = torch.Tensor

    def __getitem__(self, index):
        
        return {'img': self.type(self.img_stack[index,...]).double(), 'rvs': self.rvs_stack[index]}
    
    
    def __len__(self):
        return len(self.rvs_stack)

In [26]:
class Multi_H5_Dataset(Dataset):
    def __init__(self, h5_files):
        self.h5_files = h5_files
        self.type = torch.Tensor
        self.img_stack = np.empty((0,3,526,526))
        self.rvs_stack = np.empty((0))
        for file in h5_files:
            ds = h5.File(file,'r')
            img_stack, rvs_stack = h5_to_array(ds)
            self.img_stack = np.append(self.img_stack,img_stack,axis=0)
            self.rvs_stack = np.append(self.rvs_stack,rvs_stack,axis=0)
#         print(self.length,len(self.rvs))

    def __getitem__(self, index):
        
        return {'img': self.type(self.img_stack[index,...]).double(), 'rvs': self.rvs_stack[index]}
    
    
    def __len__(self):
        return len(self.rvs_stack)

In [27]:
# files1 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/tryagain/51Peg-3/*.h5')
# files2 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-2/*.h5')
# files3 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-3/*.h5')
# files4 = glob.glob('/scratch/mdd423/CNN_EPRV/data/HARPS/PEG51/51Peg-4/*.h5')
# file_list = files1 + files2 + files3 + files4

In [28]:
# files1

In [29]:
# dataset = Multi_H5_Dataset([files1[2]])
dataset = ND_Dataset(img_stack,bcs_stack - rvs_stack)

In [30]:
bcs_stack.shape, img_stack.shape

((311,), (311, 3, 256, 256))

In [31]:
testdata,validdata = torch.utils.data.random_split(dataset,[270,41])

In [32]:
len(dataset)

311

In [33]:
import os.path

<h2>Defining Fitting Process</h2>
including hyperparameters, the loss function, and the optimization algo

In [34]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [35]:
print(device)

cuda


In [36]:
lr = 0.001
b1 = 0.9
b2 = 0.999

In [37]:
model = RV_Model().to(device)
mse_loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

In [38]:
batch_size = 8
n_cpu = 4
dataloader = DataLoader(
    testdata,
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

validloader = DataLoader(
    validdata,
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

<h2>Training Step</h2>
the working step!

In [39]:
import sys
import time
import datetime
import itertools

In [40]:
2*len(dataset)/400

1.555

In [41]:
n_epochs = 100
train_loss = []
valid_loss = []

valiter = itertools.cycle(validloader)
b_avg = 0.0
e_avg = 0.0

start_t = time.time()

directory, tail = path.split(filename)
for j,epoch in enumerate(range(n_epochs)):
    
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        y    = model(batch['img'].to(device)).squeeze()
        loss = mse_loss(y,batch['rvs'].to(device).double())
        loss.backward()
        optimizer.step()
        b_time = time.time() - start_t
        b_avg  = b_time / (i + 1 + (j * len(dataloader)))
        
        if (i % 5) == 0:
            # Validation checkpoint every 5 batches
            valbatch = next(valiter)
            model.eval()
            with torch.no_grad():
                y     = model(     valbatch['img'].to(device)).squeeze()
                vloss = mse_loss(y,valbatch['rvs'].to(device).double())
                
                r_time = b_avg * ((len(dataloader) * n_epochs) - (i + 1 + (j * len(dataloader))))
                sys.stdout.write(
                        "\r[Epoch %d/%d] [Batch %d/%d] [Train Loss: %f] [Valid Loss: %f] [BT: %s] [ET: %s] [RT: %s]"
                        % (
                            epoch,
                            n_epochs,
                            i,
                            len(dataloader),
                            loss.item(),
                            vloss.item(),
                            str(datetime.timedelta(seconds=b_avg)),
                            str(datetime.timedelta(seconds=e_avg)),
                            str(datetime.timedelta(seconds=r_time))
                        )  
                )
            
            model.train()
            
    train_loss.append(loss.item())
    valid_loss.append(vloss.item())
    if (j % 10) == 0:
        # Saving checkpoint every 10 epoches
        modelpath = '/scratch/mdd423/CNN_EPRV/models/rv_model_{}_{}_{}_bcs-rvs.model'.format(tail[:-3],j,n_epochs)
        torch.save(model.state_dict(), modelpath)
    e_time = time.time() - start_t
    e_avg  = e_time/(j + 1)
train_loss = np.array(train_loss)
valid_loss = np.array(valid_loss)

[Epoch 0/100] [Batch 20/34] [Train Loss: 34.945680] [Valid Loss: 2814.738451] [BT: 0:00:02.938114] [ET: 0:00:00] [RT: 2:45:27.888343]]]

/ext3/miniconda3/envs/cnnenv/lib/python3.10/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Epoch 99/100] [Batch 30/34] [Train Loss: 62.761739] [Valid Loss: 6.855266] [BT: 0:00:02.311066] [ET: 0:01:18.579841] [RT: 0:00:06.933199]]]]

In [42]:
thing = 0
for parameter in model.parameters():
#     print(parameter.shape)
    thing += np.product(parameter.shape)
print(thing)

18985665


In [43]:
modelpath = '/scratch/mdd423/CNN_EPRV/models/rv_model_{}_{}_{}_bcs-rvs.model'.format(tail[:-3],j,n_epochs)
torch.save(model.state_dict(), modelpath)

In [44]:
tlname = path.join(dir_name, tailname + '_tl_bcs-rvs.nda')
vlname = path.join(dir_name, tailname + '_vl_bcs-rvs.nda')
save(tlname,train_loss)
save(vlname,valid_loss)